# TRABAJO PARCIAL: QUORIDOR



## 1. INTRODUCCION

Como trabajo parcial, para el curso de complejidad algorítmica, se implementará el juego Quoridor en el lenguaje de programación Python. El juego consiste en llegar al extremo opuesto de tablero antes que los demás jugadores, para ello los jugadores podrán poner barreras con el objetivo de bloquear los caminos de los demás jugadores y así poder llegar antes al extremo opuesto. Además, los jugadores solo pueden moverse un solo espacio (arriba, abajo, derecha o izquierda), saltar encima de una ficha, si la ficha del otro jugador esta al costado), moverse en diagonal si al querer saltar encima de una ficha existe una barrera en el siguiente espacio o poner una barrera. Para el desarrollo de este proyecto se usará Jupyter Notebook el cual se usará para explicar detalladamente el código a realizar. También se usará GitHub para poder tener un correcto control de versiones y poder trabajar de manera colaborativa. Por último, se usará la plataforma Discord para comunicarnos como equipo.

## 2. ESTADO DEL ARTE

Para el apoyo de la implementación de nuestra solución, hemos recolectado una serie de investigaciones que evidencian las posibles soluciones del juego. Por cada investigación, nos fijaremos únicamente en cómo se definió la tabla del juego y la inserción de barreras (obstáculos del juego) y qué algoritmos fueron utilizados para la implementación del pathfinding y la inserción de barreras.

### Primera solución: Monte Carlo Tree Search for Quoridor


Fuente: Massagué Respall, Victor & Brown, Joseph & Aslam, Hamna. (2018). Monte Carlo Tree Search for Quoridor. 

#### Tabla del juego 
"The class Board has all the information related to theboard game, squares, fences and both players. Allsquares are stored in an ArrayList, and to navigatethrough the board, we use a Map that each key is asquare and it returns a Set of all adjacent squares tothe key"

#### Barreras
"For checking if a certain fence can be placed, we initially start the game with a Set containing all possible placesthat a fence can be placed on the board. Then, each time a fence is placed, we update the Set removing ”forbid-den” positions. After checking that the position of that fence is inside the Set, we must check that both playersare not completely blocked from reaching the opposing baseline. The system explained before for storing fences is alsouseful for returning all possible movements available tothe player."

#### Algoritmos
"The Monte Carlo Tree Search algorithm is used forbuilding the AI agent for Quoridor, as it appears to bean eﬃcient algorithm for this type of board game andgame tree size. It is a probabilistic search algorithmwith a unique decision-making ability because of its ef-ﬁciency in open-ended environments with an enormousamount of possibilities. To deal with the size of thegame tree, it applies Monte Carlo method (Metropolisand Ulam 1949). As it is based on random sampling ofgame states, it does not need to use brute force.We have built a game tree with a root node, then it isexpanded with random simulations. In the process, wemaintain the number of times we have visited a speciﬁcnode and a win score, used to evaluate the state of theboard. In the end, we select the node with best resultsand higher win scores."

## 3. Metodología

Para el desarrollo del juego se usaran grafos el cual nos permitira saber cuales son los espacios aledaños a una ficha. Por otro lado se hara uso del algoritmo dijistra para poder encontrar el camino minimo para que la ficha llegue al extremo opuesto de donde empezo. Para ello inicialmente se usaran tableros de 3x3 con el objetivo de probar los algoritmos realizados. Luego se usaran tableros de 5x5 y 7x7 para poser vizaulizar un grafico de la complejidad.

## 4. EXPERIMENTOS 

## 5. RESULTADOS

## 6. CONCLUSIONES

## 7. PRESENTACION

# Librerias
###  Se usan la libreria pygame para la interface grafica del juego
###  se usa la libreria time para medir los tiempos de cada algoritmo implementado

In [1]:
import pygame
import time

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


# Clase Tile
### La clase tile es el nodo en cual se guardan los vecinos. Tambien tiene un metodo element el cual se inicializa vacio. En el metodo elemtendo se guarda al jugador.

In [2]:
class Tile():
    def __init__(self):
        self.neighbours = []
        self.visited = False
        self.visited_order = -1
        self.is_shortest_path = False
        element = None

# Board
### La clase board es donde se crea el tablero. Para crear el tablero se usado una matriz en la cual se guarda la clase Tile. Tambien se asigan los vecinos a cada recuadro del tablero.

In [3]:
class Board():
    def __init__(self, size):
        self.board = [[Tile() for r in range(size)] for c in range(size)]
        self.indexing_tiles(self.board)

    def indexing_tiles(self, board):
        for i in range(len(board)):
            for j in range(len(board[i])):
                if j+1 < len(board[i]):
                    board[i][j].neighbours.append(board[i][j+1])
                if j-1 >= 0:
                    board[i][j].neighbours.append(board[i][j-1])
                if i-1 >= 0:
                    board[i][j].neighbours.append(board[i-1][j])
                if i+1 < len(board):
                    board[i][j].neighbours.append(board[i+1][j])

    def print_visited_tiles(self, board):
        for i in range(len(board)):
            for j in range(len(board[i])):
                v = board[i][j].visited_order
                print(v, end = ",  " if v in range(0,9) else ", ")
            print("\n")
    
    def print_path(self, board):
        for i in range(len(board)):
            for j in range(len(board[i])):
                print(1 if board[i][j].is_shortest_path else 0, end = ",  ")
            print("\n")

    def reset_tiles(self, board):
        for i in range(len(board)):
            for j in range(len(board[i])):
                board[i][j].visited = False
                board[i][j].visited_order = -1
                board[i][j].is_shortest_path = False

# Clase Player
### La clase player define al jugador. dentro de esta clase se guarda la poscicion del jugador y se guarda la imagen que se usara para representar a ese jugador

In [4]:
class Player(pygame.sprite.Sprite):
    def __init__(self, color, xpos, ypos):
        self.color = color
        self.xpos = xpos
        self.ypos = ypos
        super().__init__()
        self.image = pygame.image.load("meteor.png").convert()
        self.image.set_colorkey((0, 0, 0))
        self.rect = self.image.get_rect()

# Clase Game
### La clase game se encarga de inicializar el juego. En esta clase se inicializan las clases player y board. Ademas, se le asigna su posicision inicial a cada jugador y el tamaño del tablero

In [5]:
class Game():
    def __init__(self, num_players, size):
        self.players = []
        colors = ['red', 'blue', 'yellow', 'green']
        xpos = [int(size/2), int(size/2), 0, size-1]
        ypos = [0, size-1, int(size/2), int(size/2)]
        self.game_board = Board(size)

        for i in range(num_players):
            self.players.append(Player(colors[i], xpos[i], ypos[i]))
            self.game_board.board[ypos[i]][xpos[i]].element = self.players[i] 
        self.all_sprite_list = pygame.sprite.Group()
    def printPlayer(self):
        for i in range(len(self.players)):
            self.players[i].rect.x = (self.players[i].xpos)*50
            self.players[i].rect.y = (self.players[i].ypos)*50
            self.all_sprite_list.add(self.players[i])

In [6]:
def find_shortest_path(tile):
    tile.is_shortest_path = True
    if tile.visited_order == 0:
        return tile
    
    posible_targets = []
    
    minimum = min(i.visited_order for i in tile.neighbours if i.visited)
    for i in tile.neighbours:
        if i.visited_order == minimum:
            posible_targets.append(i) 

    if len(posible_targets) == 1:
        neighbor_target = posible_targets[0]
    else:
        neighbors_minimum = []
        for i in posible_targets:
            neighbor_minimum = min(i.visited_order for i in tile.neighbours if i.visited)
            neighbors_minimum.append(neighbor_minimum)

        minimum = min(i for i in neighbors_minimum)
        for i in tile.neighbours:
            if i.visited_order == minimum:
                neighbor_target = i

    find_shortest_path(neighbor_target)


In [7]:
def DFS(tile_ori, tile_dest, visited_order):
    tile_ori.visited = True
    tile_ori.visited_order = visited_order
    if tile_dest.visited == True:
        return
    for i in tile_ori.neighbours: 
        if i.visited == False: 
            DFS(i, tile_dest, visited_order+1)

def call_DFS(game, pos_ori, pos_dest):
    board_util = game.game_board.board
    tile_ori = board_util[pos_ori[0]][pos_ori[1]]
    tile_dest = board_util[pos_dest[0]][pos_dest[1]]
    DFS(tile_ori, tile_dest, 0)
    find_shortest_path(tile_dest)
    game.game_board.print_visited_tiles(board_util)
    game.game_board.print_path(board_util)
    game.game_board.reset_tiles(board_util)


In [8]:
def BFS(game, pos_ori, pos_dest):
    board_util = game.game_board.board
    tile_ori = board_util[pos_ori[0]][pos_ori[1]]
    tile_dest = board_util[pos_dest[0]][pos_dest[1]]
    queque = []
    order = 0
    queque.append(tile_ori)

    tile_ori.visited = True
    while True:
        tile_ori = queque.pop(0)
        tile_ori.visited = True
        tile_ori.visited_order = order
        order += 1
        if tile_ori == tile_dest:
            break
        for i in tile_ori.neighbours:
            if queque.count(i) == 0 and i.visited == False:
                queque.append(i)
    
    find_shortest_path(tile_dest)
    game.game_board.print_visited_tiles(board_util)
    game.game_board.print_path(board_util)
    game.game_board.reset_tiles(board_util)

In [9]:
def main():
    pygame.init()
    done = False
    n = 9
    numPlayer = 4
    SCREEN_WIDTH = int((n)*50)
    SCREEN_HEIGHT = int((n)*50)
    BLACK = (0,0,0)
    all_sprite_list = pygame.sprite.Group()
    screen = pygame.display.set_mode([SCREEN_WIDTH, SCREEN_HEIGHT])
    game = Game(numPlayer, n)
    pos_ori = [1,1]
    pos_dest = [3,3]
    start = time.time()
    BFS(game, pos_ori, pos_dest)
    end = time.time()
    print(end-start)

    start = time.time()
    call_DFS(game, pos_ori, pos_dest)
    end = time.time()
    print(end-start)

    while not done:

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True

        screen.fill([255, 255, 255])
        for x in range(50,SCREEN_WIDTH,50):
            pygame.draw.line(screen,BLACK, (x,0),(x,SCREEN_WIDTH), 2)
        for y in range(50,SCREEN_HEIGHT,50):
            pygame.draw.line(screen,BLACK, (0,y),(SCREEN_HEIGHT,y), 2)

        game.printPlayer()
        game.all_sprite_list.draw(screen)
        pygame.display.flip()

    pygame.quit()



if __name__== "__main__":
    main()

8,  3,  6,  12, 18, -1, -1, -1, -1, 

2,  0,  1,  5,  11, 17, -1, -1, -1, 

9, 4,  7,  13, 19, -1, -1, -1, -1, 

15, 10, 14, 20, -1, -1, -1, -1, -1, 

-1, 16, -1, -1, -1, -1, -1, -1, -1, 

-1, -1, -1, -1, -1, -1, -1, -1, -1, 

-1, -1, -1, -1, -1, -1, -1, -1, -1, 

-1, -1, -1, -1, -1, -1, -1, -1, -1, 

-1, -1, -1, -1, -1, -1, -1, -1, -1, 

0,  0,  0,  0,  0,  0,  0,  0,  0,  

0,  1,  1,  1,  0,  0,  0,  0,  0,  

0,  0,  0,  1,  0,  0,  0,  0,  0,  

0,  0,  0,  1,  0,  0,  0,  0,  0,  

0,  0,  0,  0,  0,  0,  0,  0,  0,  

0,  0,  0,  0,  0,  0,  0,  0,  0,  

0,  0,  0,  0,  0,  0,  0,  0,  0,  

0,  0,  0,  0,  0,  0,  0,  0,  0,  

0,  0,  0,  0,  0,  0,  0,  0,  0,  

0.06979250907897949
16, 15, 14, 13, 12, 11, 10, 9, 8,  

17, 0,  1,  2,  3,  4,  5,  6,  7,  

18, 19, 20, 21, 22, 23, 24, 25, 26, 

19, 20, 21, 32, 31, 30, 29, 28, 27, 

-1, -1, -1, -1, 32, 31, 30, 29, 28, 

-1, -1, -1, -1, -1, -1, -1, -1, -1, 

-1, -1, -1, -1, -1, -1, -1, -1, -1, 

-1, -1, -1, -1, -1, -1, -1, -1, 